# 4. Procesamiento de Lenguaje Natural

Otro de los campos de aplicación para los que más se utiliza PyTorch es para el Procesamiento de Lenguaje Natural (NLP), donde se ha convertido en una de las librerías estrella para el diseño e implementación de modelos de lenguaje sofisticados, por ejemplo, para traducción automática o generación de texto. En este taller se muestran aspectos clave para comenzar tu recorrido pythonero en el mundo del NLP. 

## Antes de comenzar...

De nuevo cargamos la librería de Pytorch, y en este caso además *torchtext*, que contiene las funcionalidades necesarias para trabajar con texto con Pytorch. 

In [1]:
import torch
import torchtext
from torchinfo import summary
from torchnlp import *
from collections import Counter

## Carga del Dataset AG_NEWS

En este experimento utilizaremos el dataset AG_NEWS, que nos permitirá llevar a cabo tareas de clasificación de documentos en las categorías contempladas por el dataset: world, sports, business, sci/tech. Este dataset es muy sencillo de cargar y utilizar, puesto que viene por defecto en la librería *torchtext*.

In [2]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='../data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

## Tokenización del texto

Cuando queremos abordar tareas de procesamiento de lenguaje natural, tenemos que tomar la decisión de qué mínima unidad de representación utilizar.  Podemos trabajar, por ejemplo, a nivel de caracter, a nivel de palabra, o a nivel de frase. Esta decisión suele venir tomada por diversas razones, como puede ser las características propias del dataset, el método en concreto que queramos aplicar, y por supuesto, la tarea final en cuestión. 

En nuestro caso, hemos decidido trabajar a nivel de palabra, una de las formas más utilizadas, y que además nos permite abordar de forma sencilla la clasificación a nivel de documento. Si lo razonamos, una idea intuitiva para realizar dicha clasificación sería intentar identificar patrones/expresiones/relaciones claras a nivel de frase para determinar a qué categoría pertenece un texto. 

En nuestro caso hemos usado el tokenizador *basic_english* porque nos permite un tratamiento sencillo del texto en inglés a nivel de palabra, pero podríamos importar tokenizadores propios, de otras librerías como *Gensim* o *Spacy*, o incluso implementarlos nosotros. 

In [3]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

Al usar la clase *Counter*, podemos detectar todos los tokens que se encuentran en nuestro dataset (lo que a partir de ahora será nuestro vocabulario). Cada uno de estos tokens tiene asociada una cifra única. 

Finalmente almacenarlos en un objeto de la clase *colab*, encargada de almacenar y manejar vocabularios de un corpus. Con su método *stoi()* podemos recuperar la codificación de las palabras del vocabulario.

In [4]:
counter = Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab_ = torchtext.vocab.vocab(counter)

vocab_size = len(vocab_)

stoi = vocab_.get_stoi()

Veamos el resultado de tokenizar noticias del dataset:

In [5]:
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)


first_sentence = train_dataset[0][1]
second_sentence = train_dataset[1][1]

f_tokens = tokenizer(first_sentence)
s_tokens = tokenizer(second_sentence)

print(f'\nfirst token list:\n{f_tokens}')
print(f'\nsecond token list:\n{s_tokens}')


first token list:
['wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', 'reuters', '-', 'short-sellers', ',', 'wall', 'street', "'", 's', 'dwindling\\band', 'of', 'ultra-cynics', ',', 'are', 'seeing', 'green', 'again', '.']

second token list:
['carlyle', 'looks', 'toward', 'commercial', 'aerospace', '(', 'reuters', ')', 'reuters', '-', 'private', 'investment', 'firm', 'carlyle', 'group', ',', '\\which', 'has', 'a', 'reputation', 'for', 'making', 'well-timed', 'and', 'occasionally\\controversial', 'plays', 'in', 'the', 'defense', 'industry', ',', 'has', 'quietly', 'placed\\its', 'bets', 'on', 'another', 'part', 'of', 'the', 'market', '.']


## Codificación del texto

Para poder trabajar con entradas textuales en una red neuronal necesitamos una transformación previa que nos permita obtener una codificación numérica a partir del texto. A esta codificación numérica la llamamos **embedding** y se realiza sobre el conjunto de tokens que forma la sentencia.  

Podemos obtener una representación numérica del texto asociándole una cifra numérica a cada token de la secuencia. De esta forma, también podemos hacer la transformación inversa y obtener un texto a partir de los tokens generados y manejados por la red neuronal internamente. 

In [6]:
def encode(x):
    # print([vocab[s] for s in tokenizer(x)])
    return [stoi[s] for s in tokenizer(x)]

vec = encode(first_sentence) 

### Ejemplo

A continuación vemos un ejemplo del resultado obtenido tras emplear el tokenizador. Partimos de la siguiente sentencia: 

In [7]:
first_sentence = train_dataset[0][1]
print(first_sentence, "Sentencia")

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again. Sentencia


Los tokens que se obtienen de dicha sentencia están mostrados a continuación:

In [11]:
print(tokenizer(first_sentence), "Tokens asociados a la sentencia")

['wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', 'reuters', '-', 'short-sellers', ',', 'wall', 'street', "'", 's', 'dwindling\\band', 'of', 'ultra-cynics', ',', 'are', 'seeing', 'green', 'again', '.'] Tokens asociados a la sentencia


Y la codificación de la sentencia finalmente es:

In [13]:
vec = encode(first_sentence)
print(vec, "<- Embedding")

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 10, 12, 13, 14, 0, 15, 16, 17, 18, 19, 20, 14, 21, 22, 23, 24, 2] <- Embedding


## Modelo de bolsa de palabras

Para mostrar un ejemplo de uso de Pytorch para un modelo de lenguaje vamos a construir uno de los modelos más sencillos, conocido como **bolsa de palabras**. Tal y como la propia definición indica, estos modelos consideran las secuencias de entrada como conjuntos de palabras. En función de las palabras que forman una noticia, esta se clasifica en una o en otra clase. Es importante tener en cuenta que se trata de conjuntos de tokens que no consideran el orden original de dichos tokens en el texto. 

En nuestro caso, vamos a utilizar el conjunto de noticias que cargamos al principio del notebook para realizar un problema de **clasificación de documentos** y clasificar las noticias en función de su temática.

Podemos definir una bolsa de palabras como un vector (o tensor) de ceros donde los tokens que aparecen en el texto toman valor distinto de 0 usando para ello la posición del vector). 

In [18]:
def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(f"Sentencia:\n{train_dataset[0][1]}")
print(f"\Bolsa de palabras asociada:\n{to_bow(train_dataset[0][1])}")

Sentencia:
Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
\Bolsa de palabras asociada:
tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Ejercicio 1**: ¿Qué significa el valor 2 en la primera posición del vector?


### Entrenamiento del modelo

Ahora encapsulamos nuestros conjuntos de train y test, ya codificados anteriormente, en objetos DataLoader para proceder a entrenar el modelo. 

In [19]:
from torch.utils.data import DataLoader
import numpy as np 

def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Es un modelo muy sencillo, por lo que nuestra red se basa en una capa Sequential.

In [20]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Y una vez tenemos implementada la red, entrenamos el modelo de la forma que hemos venido haciendo a lo largo del taller. ¡Ya somos expertos!. En este caso, hemos implementado el entrenamiento para una sola época, puesto que los modelos de lenguaje son costosos computacionalmente, y este modelo además es sencillo.

In [21]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

### ¡A entrenar! 🏋🏼‍♀️

In [22]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8071875
6400: acc=0.83453125
9600: acc=0.85125
12800: acc=0.8615625


(0.02603057300104007, 0.8654717484008528)

# Pytorch Transformers

No podemos hablar de PyTorch y procesamiento de lenguaje natural sin mencionar la librería **transformers** de Hugging Face🤗.

A pesar de las grandes ventajas que da Pytorch a la hora de implementar modelos para NLP, en proyectos grandes y con redes neuronales complejas lo que se suele hacer es combinar su uso con el de la librería **transformers**. Esta librería contiene implementaciones de PyTorch de modelos de lenguaje asentados y muy utilizados actualmente, así como modelos pre-entrenados que podemos cargar directamente para su uso, ¡y mucho más! 

Aunque puedes usar Transformers por sí solo, una aplicación normalmente requiere su combinación con funcionalidades de la librería PyTorch, por ejemplo, para el uso de datasets y preprocesamiento de los datos. ¡Podrías aplicar todo lo aprendido durante este taller! 

Esta librería opera como una capa intermedia entre nuestra implementación y la librería PyTorch, permitiendo entrenar y utilizar modelos sofisticados de procesamiento de lenguaje natural en cuestión de muy pocas líneas de código. Prueba de ello es su funcionalidad *pipeline* que se encarga de cargar automáticamente todo lo necesario para ejecutar un modelo, para que únicamente te preocupes de tu tarea final.

### ¡Veamos un ejemplo!
Podemos usar *pipeline* con un modelo de clasificación de sentimientos. Con tres líneas hemos podido saber si un texto contiene connotaciones positivas o negativas. 😳🤯

In [28]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I am super excited because I've been waiting for a Python Conference in Granada my whole life.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9984990358352661}]

In [29]:
classifier("I've been waiting for a Python Conference in Granada my whole life. Miguel and Andrea are really disgusting people.")

[{'label': 'NEGATIVE', 'score': 0.9980828762054443}]

> **Ejercicio 2**: elige otro problema de procesamiento de lenguaje natural, e intenta resolverlo con pipeline! 
PyAyuda: Puedes probar con'question-answering' o 'summarization' si no se te ocurre nada! 😉